# Chapter 5 Exercises

1. What is the fundamental idea behind support vector machines? 
 - SVMs aim to create the widest possible street or margin between two groups. They use the data points from each group which are closest to the other group (support vectors) to define this margin.
 - Goal is to have the widest possible margin between the decision boundary and the training instances. 
 - Also important to use kernels for training on nonlinear datasets. 
2. What is a support vector? 
 - support vectors are the data points at the boundary between two groups of data. They are used to define the border between the two groups and calculate predictions. 
 - Instances which are not on the margin do not affect the training or predictions. 
3. Why is it important to scale the inputs when using SVMs?
 - SVMs ultimately rely upon measuring the distance between points in order to create the widest possible margin between groups. Having features of vastly different scales distorts the distances between points and overweights the feature with the larger scale. 
4. Can an SVM output a confidence score when it classifies an instance? What about a probability? 
 - The output of the decision function or the distance between an instance and the decision boundary can be used as a confidence score. SVMs do not output a probability for each class. 
 - Using logisitic regression and cross validation, sklearn can calculate a probability for each instance when using an SVM. 
5. Should you use the primal or the dual form of the SVM problem to train a model on a training set with millions of instances and hundreds of features? 
 - You should use the primal form. The dual form is faster when the number of features is greater than the number of instances. 
 - nonlinear (kernelized) SVMs can only use the dual form. 
 - computational complexity for the primal form is directly proportional to the number of instances m, while for the dual form it is an exponential relationship, $m^2$ or $m^3$
6. Say you've trained an SMV classifier with an RBF kernel, but it seems to underfit the training set. Should you increase or decrease gamma? What about C?
 - Increase gamma to decrease each instances range of influence on the decision boundary. This will introduce more variation in the decision boundary and reduce underfitting. Alternatively you could also increase C. 
7. How should you set the QP parameters to solve the soft margin linear SVM classifier problem using an off the shelf QP solver? 
 - H an identity matrix of size 2m by 2m + 1 where m equals the number of instances. Top left value is zero to ignore the bias term. 
 - f a vector of zeros with length = # features + 1 for bias term and concatenated with a vector with each element = C of length = #instances
 - A a matrix of size #constraints x #features+1 plus an mxm identity matrix to the right, an inverse identity matrix below that, and zeros in the remainder of the array. 
 - b is a vector of -1s of length equal to the number of training instances concatenated with a vector of 0s equal in length to #instances

In [3]:
# 8. Train a linear SVC on a linearly separable dataset. 
#Then train an SVC and a SGDClassifier on the same dataset. 
#See if you can get them to produce roughly the same model. 

from sklearn import datasets

iris = datasets.load_iris()
X = iris["data"][:, (2, 3)]  # petal length, petal width
y = iris["target"]

setosa_or_versicolor = (y == 0) | (y == 1)
X = X[setosa_or_versicolor]
y = y[setosa_or_versicolor]

In [ ]:
# 9. Train an SVM classifier on the MNIST dataset. Since SVM classifiers are binary classifiers, 
# you will need to use one versus the rest to classifiy all 10 digits.
# Tune the hyperparameters using small validation sets to speed up the process. 
# What accuracy can you reach?


In [ ]:
# 10. Train an SVM Regressor on the California Houseing dataset.